In [88]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.ndimage as ndimage
import scipy.io as sio
import pandas as pd
from skimage import metrics
from tqdm import tqdm

In [53]:
images = mat73.loadmat(r'Datasets/VisualMapTest/images_CCW1Mesh_visualmap.mat')
data = mat73.loadmat(r'Datasets/VisualMapTest/data_CCW1Mesh_visualmap.mat')

In [54]:
noisy_images = images['noisy_images']
all_x = data['all_x']
all_y = data['all_y']
all_z = data['all_z']

xgrid = np.arange(-79,81,2)#-79:2:79;
ygrid = np.arange(-119,-59,2)#-119:2:-57;
zgrid = np.arange(-58,70,2)#-58:2:68;

locations = np.array([all_x, all_y, all_z])

In [55]:
noisy_image = noisy_images[:,:,:, 4]
noisy_image.shape

(32, 80, 64)

In [56]:
def coordToIndex(coord):
    return np.argwhere(ygrid==coord[1])[0][0], np.argwhere(xgrid==coord[0])[0][0], np.argwhere(zgrid==coord[2])[0][0]


def indexToCoord(index):
    if len(np.array(index).shape) == 2:
        return np.array([xgrid[index[:,1]], ygrid[index[:,0]], zgrid[index[:,2]]]).T
    else:
        return np.array([xgrid[index[1]], ygrid[index[0]], zgrid[index[2]]]).T

## Guy Perkins implementation

In [57]:
def LOCAscore(image, target_point):
    """
    Distance from the max value of the image to the target point
    :param image: 
    :param target_point: 
    :return: 
    """
    ind = np.unravel_index(np.argmax(image, axis=None), image.shape)
    image_point = indexToCoord(ind)
    localisation_error = np.linalg.norm(image_point - target_point)
    return localisation_error


def LOCA(processed_images, target_points):
    LOCAimage = np.zeros(processed_images[...,0].shape)
    for i in tqdm(range(processed_images.shape[3])):
        score = LOCAscore(processed_images[...,i], target_points[:,i])
        LOCAimage[coordToIndex(target_points[...,i])] = score
    return LOCAimage

## White and Culver implementation

In [144]:
def LOCAscoreWhite(image, target_point):
    """
    Distance from the centre of mass of the activation to the target point
    :param image: 
    :param target_point: 
    :return: 
    """
    image_max = np.max(image)
    ind = ndimage.center_of_mass(np.where(image>image_max/2, image, 0))
    localisation_error = np.linalg.norm(np.array(ind) - coordToIndex(target_point))
    return localisation_error * 2 #2mm voxels


def LOCAWhite(processed_images, target_points):
    LOCAimage = np.zeros(processed_images[...,0].shape)
    for i in tqdm(range(processed_images.shape[3])):
        score = LOCAscoreWhite(processed_images[...,i], target_points[:,i])
        LOCAimage[coordToIndex(target_points[...,i])] = score
    return LOCAimage

In [292]:
def FWHMscore(image):
    """
    Maximum distance between the two nodes that are more than or equal to 50% of the max reconstruction
    :param image: 
    :return: 
    """
    image_max = np.max(image)
    idxs = np.argwhere(image > image_max / 2)
    score = 0
    for idx in idxs:
        newscore = np.max(np.linalg.norm(idxs - idx, axis=1)) * 2  #2mm voxels
        if newscore > score:
            score = newscore
    return score


def FWHM(processed_images, target_points):
    FWHM_image = np.zeros(processed_images[..., 0].shape)
    for i in tqdm(range(processed_images.shape[3])):
        score = FWHMscore(processed_images[..., i])
        FWHM_image[coordToIndex(target_points[..., i])] = score
    return FWHM_image

    image_max = np.max(image)
    idxs = np.argwhere(image>image_max/2)
    big_idxs = np.broadcast_to(idxs, [idxs.shape[0],idxs.shape[0],idxs.shape[1]])
    score = np.max(np.linalg.norm(np.swapaxes(big_idxs, 0, 1) - idxs, axis=2))
    return score * 2 #2mm voxels


def FWHM(processed_images, target_points):
    FWHM_image = np.zeros(processed_images[...,0].shape)
    for i in tqdm(range(processed_images.shape[3])):
        score = FWHMscore(processed_images[...,i])
        FWHM_image[coordToIndex(target_points[...,i])] = score
    return FWHM_image

In [272]:
def ERESscore(image, target_point):
    """
    Distance from the target point to the furthest value greater than 50% of the max value
    :param image: 
    :param target_point: 
    :return: 
    """
    image_max = np.max(image)
    idxs = np.argwhere(image>image_max/2)
    effective_resolution = np.max(np.linalg.norm(indexToCoord(idxs) - target_point, axis=1)) * 2
    return effective_resolution


def ERES(processed_images, target_points):
    ERES_image = np.zeros(processed_images[...,0].shape)
    for i in tqdm(range(processed_images.shape[3])):
        score = ERESscore(processed_images[...,i], target_points[:,i])
        ERES_image[coordToIndex(target_points[...,i])] = score
    return ERES_image

## Testing

In [273]:
test_truth = np.zeros((10,10,10))
test_truth[2,2,2] = 1
test_truth[2,1,1] = 1

In [274]:
LOCAscoreWhite(test_truth, indexToCoord([2,2,1]))

1.4142135623730951

In [275]:
FWHMscore(test_truth)

2.8284271247461903

In [276]:
ERESscore(test_truth, indexToCoord([2,2,3]))

8.94427190999916

## Processing data

In [277]:
unsmoothed_images = images['noisy_images']
unet_images = images['recon2']
old_unet_images = images2['recon2']
smooth_images = images['smooth_images']

KeyError: 'recon2'

In [284]:
unsmoothed_LOCA = LOCAWhite(unsmoothed_images, locations)
unet_LOCA = LOCAWhite(unet_images, locations)
old_unet_LOCA = LOCAWhite(old_unet_images, locations)
smooth_LOCA = LOCAWhite(smooth_images, locations)

100%|██████████| 10/10 [00:00<00:00, 458.64it/s]


NameError: name 'unet_images' is not defined

In [285]:
unsmoothed_FWHM = FWHM(unsmoothed_images, locations)
unet_FWHM = FWHM(unet_images, locations)
old_unet_FWHM = FWHM(old_unet_images, locations)
smooth_FWHM = FWHM(smooth_images, locations)

100%|██████████| 10/10 [00:00<00:00, 214.16it/s]


NameError: name 'unet_images' is not defined

In [286]:
unsmoothed_ERES = ERES(unsmoothed_images, locations)
unet_ERES = ERES(unet_images, locations)
old_unet_ERES = ERES(old_unet_images, locations)
smooth_ERES = ERES(smooth_images, locations)

100%|██████████| 10/10 [00:00<00:00, 3799.88it/s]


NameError: name 'unet_images' is not defined

In [ ]:
sio.savemat('visualmap.mat', {'unsmoothed_LOCA': unsmoothed_LOCA, 'unet_LOCA': unet_LOCA, 'old_unet_LOCA': old_unet_LOCA, 'smooth_LOCA': smooth_LOCA, 'unsmoothed_FWHM': unsmoothed_FWHM, 'unet_FWHM': unet_FWHM, 'old_unet_FWHM': old_unet_FWHM, 'smooth_FWHM': smooth_FWHM, 'unsmoothed_ERES': unsmoothed_ERES, 'unet_ERES': unet_ERES, 'old_unet_ERES': old_unet_ERES,'smooth_ERES': smooth_ERES})

In [ ]:
thing = smooth_ERES

colors = np.empty(thing.shape, dtype='object')
colors[thing < np.quantile(thing[thing != 0], 0.25)] = 'purple'
colors[thing > np.quantile(thing[thing != 0], 0.25)] = 'blue'
colors[thing > np.quantile(thing[thing != 0], 0.5)] = 'green'
colors[thing > np.quantile(thing[thing != 0], 0.75)] = 'red'

ax = plt.figure().add_subplot(projection='3d')
ax.voxels(thing, facecolors=colors)
ax.set_aspect('equal')

plt.show()

In [154]:
(np.arange(1,4) + np.ones([2]))

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [177]:
l = np.arange(1,4)
np.broadcast_to(l, [l.shape,l.shape])


TypeError: '<' not supported between instances of 'tuple' and 'int'

In [158]:
np.ones([2])

array([1., 1.])